# AUTOR : SERGIO CAÑON

## TAREA FINAL: Datos de vivienda

<p> El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. 
Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio </p>

<!-- <p> En este caso vamos a trabajar con una tabla que contienen información de distintos inmuebles repartidos por una zona específica, sus carácterísticas y su precio </p>  -->

Todas las cuestiones se deben realizar sobre el conjunto de casos que representen viviendas ubicadas en zonas residenciales **(alta, media y baja densidad)**

Las variables de las que se compone el dataset son:

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

 ## INTENTO IMPORTACIÓN EXCEL A SPARK

Primero he instalado Scala que es el lenguaje en el que originalmente fue diseñado Spark

In [ ]:
%%shell
SCALA_VERSION=2.12.8 ALMOND_VERSION=0.3.0+16-548dc10f-SNAPSHOT
curl -Lo coursier https://git.io/coursier-cli
chmod +x coursier
./coursier bootstrap \
    -r jitpack -r sonatype:snapshots \
    -i user -I user:sh.almond:scala-kernel-api_$SCALA_VERSION:$ALMOND_VERSION \
    sh.almond:scala-kernel_$SCALA_VERSION:$ALMOND_VERSION \
    --sources --default=true \
    -o almond-snapshot --embed-files=false
rm coursier
./almond-snapshot --install --global --force
rm almond-snapshot

Configuro el Shell y recargo Colab para que funcione Scala

In [ ]:
%%shell
echo "{
  \"language\" : \"scala\",
  \"display_name\" : \"Scala\",
  \"argv\" : [
    \"bash\",
    \"-c\",
    \"env LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libpython3.6m.so:\$LD_PRELOAD java -jar /usr/local/share/jupyter/kernels/scala/launcher.jar --connection-file {connection_file}\"
  ]
}" > /usr/local/share/jupyter/kernels/scala/kernel.json

A continuación descargo e instalo com.crealytics que es lo equivalente a una librería en Spark para leer fichero excel

In [ ]:
import $ivy.`com.crealytics:spark-excel_2.12:0.13.1`


La siguiente parte no llegué ha conseguir hacerlo puestp qeu deduzco qeu me faltaba crear un contexto Saprk con Scala. Aún asi, para la versión instalada de Scala (2.12.8) habría qeu ejecutar lo siguiente:

In [ ]:
import org.apache.spark.sql._

val spark: SparkSession = ???
val df = spark.read
    .format("com.crealytics.spark.excel")
    .option("dataAddress", "'My Sheet'!B3:C35") // Optional, default: "A1"
    .option("header", "true") // Required
    .load("BDpracticafinalCSV.xlsx")

## Inicializar y cargar el contexto pyspark

Instalamos Spark 3.0.1

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

import findspark
findspark.init()
# en primer lugar inicio el contexto de Spark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# SPAR SQL
from pyspark.sql import *
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.sql.types import FloatType


 

*   Importo el fichero 
*   Extraigo el el nombre de cada columna y el nombre RDD para filtrar y limpiar los RD sin el nombre de las columnas .
*    Hago un paseo y vuelvo a filtrar por RM RL RH añadiendo la función Upper() para limpiar los nombres mal escritos


 

In [ ]:
# IMPORTACIÓN, FILTRADO Y LIMPIEZA

vivienda_RDD = sc.textFile("BDpracticafinalCSV.csv")
vivienda_RDD_top = vivienda_RDD.take(1)[0]
vivienda_RDD = vivienda_RDD.filter(lambda x: x != vivienda_RDD_top)
vivienda_RDD = vivienda_RDD.map(lambda x: x.split(";"))
vivienda_RDD = vivienda_RDD.filter(lambda x: x[1].upper() in ["RM","RL","RH"]).map(tuple)

In [ ]:
print("NUMERO DE VALORES", vivienda_RDD.count())
print("ASPECTO RDD", vivienda_RDD.first())

NUMERO DE VALORES 2768
ASPECTO RDD ('1', 'RL', '141', '31770', 'Lvl', 'Gtl', '6', '1960', '1960', 'Stone', 'TA', 'Gd', '1080', 'Fa', 'Y', '1', '0', '3', '1', 'TA', '7', '2', '528', 'TA', '0', 'Sp', '5', '2010', '215000')


## Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?


*   Selecciono la fila de Order y añado un 1, para hacer un reduceByKey y saber cuantos Order repetidos hay
*   Filtro los que haya más de 1 y los localizo en Order
*   Importo el dataset con el df para usar dropDuplicates. Uso "subset" para eliminar estrictamente los duplicados de Order
*   Convierto el nuevo df en RDD
*   Compruebo con count() que hay menos valores

In [ ]:
viviendas_diferentes = vivienda_RDD.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b).map(lambda x: (float(x[1]), x[0])).sortByKey(ascending=False)
viviendas_duplicadas = viviendas_diferentes.filter(lambda x: x[0]!=1).map(lambda x: x[1])
print("VIVIENDAS DUPLICADAS:", viviendas_duplicadas.collect())

VIVIENDAS DUPLICADAS: ['2898', '2899', '2900', '2901', '2930', '2929']


In [ ]:
vivienda_DF = spark.read.csv('BDpracticafinalCSV.csv',header=True, inferSchema='true', sep=';')
vivienda_DF.show(5)

+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|Order|MS Zoning|Lot Frontage|Lot Area|Land Contour|Land Slope|Overall Qual|Year Built|Year Remod/Add|Mas Vnr Type|Exter Qual|Bsmt Cond|Total Bsmt SF|Heating QC|Central Air|Full Bath|Half Bath|Bedroom AbvGr|Kitchen AbvGr|Kitchen Qual|TotRms AbvGrd|Garage Cars|Garage Area|Garage Cond|Pool Area|Pool QC|Mo Sold|Yr Sold|SalePrice|
+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|    1|      

In [ ]:
vivienda_DF = vivienda_DF.dropDuplicates(subset=[c for c in vivienda_DF.columns if c == 'Order'])


In [ ]:
vivienda_RDD = vivienda_DF.rdd.map(tuple)
vivienda_RDD = vivienda_RDD.filter(lambda x: x[1].upper() in ["RM","RL","RH"]).map(tuple)
vivienda_RDD.count()

2762

## ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?


*   Selecciono la fila de  ZONA y añado un 1, para hacer un reduceByKey para contar cuátos hay
*   Para el precio, extraigo la fila de SalePrice, hago un join con la varible anteriomente creada y calculo la media redondeando


In [ ]:
# TOTAL DE INMUEBLES
 
numero_por_zona = vivienda_RDD.map(lambda x: (x[1].upper(), 1)).reduceByKey(lambda a, b: a + b)
numero_por_zona.take(15)

[('RL', 2273), ('RH', 27), ('RM', 462)]

In [ ]:
# MEDIA PRECIO DE VENTA

sumatorio_precio = vivienda_RDD.map(lambda x: (x[1].upper(), float(x[28]))).reduceByKey(lambda a, b: a + b)
sumatorio_precio.collect()

[('RL', 436229331.0), ('RH', 3683334.0), ('RM', 58573004.0)]

In [ ]:
precio_medio_zona = sumatorio_precio.join(numero_por_zona)
precio_medio_zona = precio_medio_zona.map(lambda x:  (x[0], round(x[1][0]/x[1][1],2) )).sortByKey() # media
print("PRECIO MEDIO POR ZONA:" ,precio_medio_zona.collect())

PRECIO MEDIO POR ZONA: [('RH', 136419.78), ('RL', 191412.61), ('RM', 126781.39)]


## Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built. 
## ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

* En este paso a crear un nueva clave qué consiste en el que en las dos primeras cifras es el siglo y la tercera cifra es la década correspondiente a ese siglo 

### Total Bsmt SF
 * He seguido el mísmo método para la media con ReduceBykey.
 * El resultado ha sido que las viviendas más grandes son las de la decada de los 10 en el siglo xxi
 * además se ve que cada decada se hacen casas más grandes

In [ ]:
decada_numero_total = vivienda_RDD.map(lambda x: (str(x[7])[0:3], 1)).reduceByKey(lambda a, b: a + b)

In [ ]:
decada_numero_total.take(5)

[('198', 120), ('192', 190), ('190', 36), ('189', 12), ('188', 8)]

In [ ]:
decada_sumatorio_bsmt = vivienda_RDD.map(lambda x: (str(x[7])[0:3], float(x[12]))).reduceByKey(lambda a, b: a + b)
decada_sumatorio_bsmt.take(3)

[('198', 130320.0), ('192', 158448.0), ('190', 25925.0)]

In [ ]:
bsmt_medio_decada = decada_sumatorio_bsmt.join(decada_numero_total)
bsmt_medio_decada = bsmt_medio_decada.map(lambda x:  (x[0], round(x[1][0]/x[1][1],2) )).sortByKey() # media
bsmt_medio_decada.collect()

[('187', 761.0),
 ('188', 836.0),
 ('189', 885.58),
 ('190', 720.14),
 ('191', 788.3),
 ('192', 833.94),
 ('193', 769.92),
 ('194', 706.93),
 ('195', 968.3),
 ('196', 1082.67),
 ('197', 953.92),
 ('198', 1086.0),
 ('199', 1138.6),
 ('200', 1349.77),
 ('201', 1539.0)]

### Heating QC

* He creado una función para hacer una escala numerica con el ranking predeterminado y así mejorar la interpretabilidad de los resultados.

* Había cinco posibles casos por lo qeu la escala es del 1 al 5.

* Como era de esperar, a medida que pasan las décadas, mejor es de media la calidad de la calefaccion.

In [ ]:
vivienda_RDD.map(lambda x: x[13]).distinct().collect()

['Fa', 'Po', 'TA', 'Ex', 'Gd']

In [ ]:
def heating_transformacion(x):
  if x == "Ex":
    return 5
  elif x == "Gd":
    return 4
  elif x == "TA":
    return 3
  elif x =="Fa":
    return 2
  else:
    return 1  

In [ ]:
decada_sumatorio_heating = vivienda_RDD.map(lambda x: (str(x[7])[0:3], heating_transformacion(x[13]))).reduceByKey(lambda a, b: a + b)
decada_sumatorio_heating.take(3)

[('198', 450), ('192', 693), ('190', 142)]

In [ ]:
heating_medio_decada = decada_sumatorio_heating.join(decada_numero_total)
heating_medio_decada = heating_medio_decada.map(lambda x:  (x[0], round(x[1][0]/x[1][1],2) )).sortByKey() # media
heating_medio_decada.collect()

[('187', 3.0),
 ('188', 3.88),
 ('189', 4.0),
 ('190', 3.94),
 ('191', 3.92),
 ('192', 3.65),
 ('193', 3.92),
 ('194', 3.83),
 ('195', 3.75),
 ('196', 3.69),
 ('197', 3.45),
 ('198', 3.75),
 ('199', 4.72),
 ('200', 4.96),
 ('201', 5.0)]

## ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

* Primero, filtro los años. 
* Segundo, he hecho la conversión de pies cuadrados a metros cuadrados multiplicando por 0.092
* Ordeno ascending=False para hacer un take y seleccionar el top 10
* En el resultado aparecen "(precio por metro cuadrado, order)"

In [ ]:
precio_metro_cuadrado = vivienda_RDD.filter(lambda x: float(x[27])==2009).map(lambda x: ((float(x[28])/(float(x[3])*0.092903),x[0]))).sortByKey(ascending=False).take(10)
print("TOP 10 DEL 2009, PRECIO METRO CUADRADO Y SU ORDER")
precio_metro_cuadrado

TOP 10 DEL 2009, PRECIO METRO CUADRADO Y SU ORDER


[(1026.7118971897073, '936'),
 (991.0325013521444, '934'),
 (821.0570122815511, '464'),
 (807.9613655043873, '935'),
 (765.647528938273, '408'),
 (756.0368904997173, '407'),
 (717.5943367454944, '405'),
 (714.281725843034, '933'),
 (711.1872444531239, '403'),
 (701.0707829388548, '411')]

## Media anual por zonas del precio de venta y metros cuadrados.

*  Aqui he filtrado por zona y de cada zona calculo por separado la media del precio y de los metros cuadrados
* Filtro con upper() para evitar valores mal escritos
* Al final hago un print() de los resultados e interpreto

In [ ]:
numero_por_zona.take(5)

[('RL', 2279), ('RH', 27), ('RM', 462)]

## RL

In [ ]:
# PRECIO MEDIO

RL_sumatorio = vivienda_RDD.filter(lambda x: x[1].upper() == "RL").map(lambda x: (x[27], float(x[28]))).reduceByKey(lambda a, b: a + b)
RL_numero_total = vivienda_RDD.filter(lambda x: x[1].upper() == "RL").map(lambda x: (x[27], 1)).reduceByKey(lambda a, b: a + b)
RL_media_por_año = RL_sumatorio.join(RL_numero_total) 
RL_media_por_año = RL_media_por_año.map(lambda x: (x[0], round(x[1][0]/x[1][1],2) )).sortByKey() # media
RL_media_por_año = RL_media_por_año.collect()


In [ ]:
# METROS CUADRADOS

RL_sumatorio = vivienda_RDD.filter(lambda x: x[1].upper() == "RL").map(lambda x: (x[27], float(x[3])*0.092903)).reduceByKey(lambda a, b: a + b)
RL_numero_total = vivienda_RDD.filter(lambda x: x[1].upper() == "RL").map(lambda x: (x[27], 1)).reduceByKey(lambda a, b: a + b)
RL_media_por_año_metros = RL_sumatorio.join(RL_numero_total) 
RL_media_por_año_metros = RL_media_por_año_metros.map(lambda x: (x[0], round(x[1][0]/x[1][1],2) )).sortByKey() # media
RL_media_por_año_metros = RL_media_por_año_metros.collect()


## RM


In [ ]:
# PRECIO MEDIO

RM_sumatorio = vivienda_RDD.filter(lambda x: x[1].upper() == "RM").map(lambda x: (x[27], float(x[28]))).reduceByKey(lambda a, b: a + b)
RM_numero_total = vivienda_RDD.filter(lambda x: x[1].upper() == "RM").map(lambda x: (x[27], 1)).reduceByKey(lambda a, b: a + b)
RM_media_por_año = RM_sumatorio.join(RM_numero_total) 
RM_media_por_año = RM_media_por_año.map(lambda x: (x[0], round(x[1][0]/x[1][1],2) )).sortByKey() # media
RM_media_por_año = RM_media_por_año.collect()

In [ ]:
# METROS CUADRADOS

RM_sumatorio = vivienda_RDD.filter(lambda x: x[1].upper() == "RM").map(lambda x: (x[27], float(x[3])*0.092903)).reduceByKey(lambda a, b: a + b)
RM_numero_total = vivienda_RDD.filter(lambda x: x[1].upper() == "RM").map(lambda x: (x[27], 1)).reduceByKey(lambda a, b: a + b)
RM_media_por_año_metros = RM_sumatorio.join(RM_numero_total) 
RM_media_por_año_metros = RM_media_por_año_metros.map(lambda x: (x[0], round(x[1][0]/x[1][1],2) )).sortByKey() # media
RM_media_por_año_metros = RM_media_por_año_metros.collect()

## RH

In [ ]:
# PRECIO MEDIO

RH_sumatorio = vivienda_RDD.filter(lambda x: x[1].upper() == "RH").map(lambda x: (x[27], float(x[28]))).reduceByKey(lambda a, b: a + b)
RH_numero_total = vivienda_RDD.filter(lambda x: x[1].upper() == "RH").map(lambda x: (x[27], 1)).reduceByKey(lambda a, b: a + b)
RH_media_por_año = RH_sumatorio.join(RH_numero_total) 
RH_media_por_año = RH_media_por_año.map(lambda x: (x[0], round(x[1][0]/x[1][1],2) )).sortByKey() # media
RH_media_por_año = RH_media_por_año.collect()

In [ ]:
# METROS CUADRADOS

RH_sumatorio = vivienda_RDD.filter(lambda x: x[1].upper() == "RH").map(lambda x: (x[27], float(x[3])*0.092903)).reduceByKey(lambda a, b: a + b)
RH_numero_total = vivienda_RDD.filter(lambda x: x[1].upper() == "RH").map(lambda x: (x[27], 1)).reduceByKey(lambda a, b: a + b)
RH_media_por_año_metros = RH_sumatorio.join(RH_numero_total) 
RH_media_por_año_metros = RH_media_por_año_metros.map(lambda x: (x[0], round(x[1][0]/x[1][1],2) )).sortByKey() # media
RH_media_por_año_metros = RH_media_por_año_metros.collect()

### COMPARATIVA PRECIO

In [ ]:
print("RH",RH_media_por_año)
print("RM",RM_media_por_año)
print("RL",RL_media_por_año)

RH [('2006', 140316.67), ('2007', 147250.0), ('2008', 150333.33), ('2009', 116206.8), ('2010', 126280.0)]
RM [('2006', 127502.64), ('2007', 128947.62), ('2008', 133336.12), ('2009', 118759.87), ('2010', 119681.71)]
RL [('2006', 194911.28), ('2007', 193399.98), ('2008', 189885.59), ('2009', 190455.55), ('2010', 184978.42)]


### COMPARATIVA METROS

In [ ]:
print("RH",RH_media_por_año_metros)
print("RM",RM_media_por_año_metros)
print("RL",RL_media_por_año_metros)

RH [('2006', 659.56), ('2007', 651.32), ('2008', 766.14), ('2009', 647.14), ('2010', 753.03)]
RM [('2006', 575.01), ('2007', 625.45), ('2008', 623.67), ('2009', 552.56), ('2010', 586.1)]
RL [('2006', 1043.69), ('2007', 1059.68), ('2008', 1047.65), ('2009', 1002.18), ('2010', 1011.36)]


### **INTERPRETACION**
* Las residencias en zonas de alta densidad son las que siben el precio más a lo largo de los años mientras las otras o se mantienen o se reducen

## ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

* Primero veo los diferentes tipos de valores
* Con Stone calculo la suma de los precios con reducebykey para ver la facturacion
* Con ladrillos, extraigo los tres primeros caracteres que son los que me indican que está hecho de ladrillo (brk)
* Se ve qeu la facturación es mayor en las casas de ladrillos. Está más demandada

In [ ]:
vivienda_RDD.map(lambda x: x[9]).distinct().collect()

['None', '', 'CBlock', 'Stone', 'BrkFace', 'BrkCmn']

In [ ]:
stone_sumatorio = vivienda_RDD.filter(lambda x: x[9] == "Stone").map(lambda x: (x[9], float(x[28]))).reduceByKey(lambda a, b: a + b)
stone_sumatorio.take(3)

[('Stone', 60495477.0)]

In [ ]:
BrkFace_sumatorio = vivienda_RDD.map(lambda x: (x[9][0:3],float(x[28]))).filter(lambda x : x[0]=="Brk").reduceByKey(lambda a, b: a + b)
BrkFace_sumatorio.take(3)

[('Brk', 182918036.0)]

## ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

* Primero veo en los Garajes las opciones que puede haber y las filtro como string para luego filtrar solamente por 2 el de las cocinas. Lo convierto a Float para poderlo pasar a  dataframe y añadir FloatType()
* Segundo, hago lo mismo con la otras dos condiciones
* Tercero aplico una versión de una fórmula que hay en el libro "Learning Pyspark"
* Cuarto, dejo la interpretacion de los resultados abajo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
vivienda_RDD.map(lambda x: x[21]).distinct().collect() #GARAJE

['1', '0', '4', '', '2', '3', '5']

In [ ]:
dos_cocinas_dos_o_mas_plazas = vivienda_RDD.filter(lambda x: x[21] in ["2","3","4","5"]).filter(lambda x: x[18]=="2").map(lambda x: float(x[28]))
una_cocina_una_plaza = vivienda_RDD.filter(lambda x: x[21]=="1").filter(lambda x: x[18]=="1").map(lambda x: float(x[28]))

In [ ]:
una_cocina_una_plaza_df = spark.createDataFrame(una_cocina_una_plaza, FloatType())
dos_cocinas_dos_o_mas_plazas_df = spark.createDataFrame(dos_cocinas_dos_o_mas_plazas, FloatType())

In [ ]:
dos_cocinas_dos_o_mas_plazas_df.show(5)

+--------+
|   value|
+--------+
|112000.0|
|154300.0|
|135000.0|
|144000.0|
|122500.0|
+--------+
only showing top 5 rows



In [ ]:
def describe_pd(df_in, columns, deciles=False):
  percentiles = [25, 50, 75]
  percs = np.transpose([np.percentile(df_in.select(x).collect(),percentiles) for x in columns])
  percs = pd.DataFrame(percs, columns=columns)
  percs['summary'] = [str(p) + '%' for p in percentiles]
  spark_describe = df_in.describe().toPandas()
  new_df = pd.concat([spark_describe, percs],ignore_index=True)
  new_df = new_df.round(2)
  return new_df[['summary'] + columns]

In [ ]:
""" Function to union the basic stats results and deciles
:param df_in: the input dataframe
:param columns: the cloumn name list of the numerical variable
:param deciles: the deciles output
:return : the numerical describe info. of the input dataframe
:author: Ming Chen and Wenqiang Feng
:email: von198@gmail.com 
"""

### **INTERPRETACION**
* Se valora muy positivamente tener mas cocinas y plazas de garaje, además se consideran casas de lujo porqeue sólo hay 86 frente a las 755 del resto
* Sin embargo, la casa con el precio máximo pertenece al grupo de "UNA COCINA Y UNA PLAZA". Habria qeu ver si es porqeu compensa con otros atributos

In [ ]:
print("DOS PLAZAS O MÁS Y DOS COCINAS")
print(describe_pd(dos_cocinas_dos_o_mas_plazas_df,["value"]))
print("-----------------------------------------------------")
print("-----------------------------------------------------")
print("UNA COCINA Y UNA PLAZA")
print(describe_pd(una_cocina_una_plaza_df,["value"]))

DOS PLAZAS O MÁS Y DOS COCINAS
  summary               value
0   count                  86
1    mean  145124.27906976745
2  stddev   40555.86827593909
3     min             81400.0
4     max            269500.0
5     25%              118973
6     50%              141800
7     75%              158500
-----------------------------------------------------
-----------------------------------------------------
UNA COCINA Y UNA PLAZA
  summary               value
0   count                 755
1    mean  128121.99337748345
2  stddev  30303.813376544425
3     min             35000.0
4     max            330000.0
5     25%              111625
6     50%              128000
7     75%              142062
